In [1]:
# Colab cell 1 — install deps (run once)
!apt-get update -qq
# tesseract for OCR
!apt-get install -y -qq tesseract-ocr libtesseract-dev
# Python packages
!pip install -q ultralytics roboflow opencv-python-headless pytesseract


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Selecting previously unselected package libleptonica-dev.
Preparing to unpack .../libleptonica-dev_1.82.0-3build1_amd64.deb ...
Unpacking libleptonica-dev (1.82.0-3build1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack .../libtesseract-dev_4.1.1-2.1build1_amd64.deb ...
Unpacking libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Setting up libleptonica-dev (1.82.0-3build1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Processing triggers for ma

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aOtqU9FLNddzGyowbT6x")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-11 in yolov8:: 100%|██████████| 20262/20262 [00:03<00:00, 6739.70it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
# Colab cell 2 — configuration (edit this)
import os

# --- Option A: automatic Roboflow download (preferred) ---
# If you have a Roboflow API key, paste it here. Otherwise leave blank and use Option B below.
ROBOFLOW_API_KEY = "aOtqU9FLNddzGyowbT6x"  # <- paste your Roboflow API key here if you have one

# The Roboflow "workspace/project" slug for your dataset:
# From your link the universe project slug: `roboflow-universe-projects/license-plate-recognition-rxg4e`
# Roboflow API uses workspace/project/version format - you may need to adapt if you exported your own.
# If using API set PROJECT = "<workspace>/<project>" and VERSION = "11"
PROJECT = "roboflow-universe-projects/license-plate-recognition-rxg4e"
VERSION = "11"

# Local paths
DATA_DIR = "/content/License-Plate-Recognition-11"   # where dataset will be extracted
YOLO_DATA_YAML = "data.yaml"         # YAML for ultralytics
OUTPUT_DIR = "runs"                  # ultralytics default runs dir

# Training settings
EPOCHS = 15
BATCH = 16
MODEL_NAME = "yolov8n.pt"  # small; change to yolov8s.pt, yolov8m.pt if you want

# Video test settings
INPUT_VIDEO = "input.mp4"   # if you upload a video to Colab, name it like this (or change path)
OUTPUT_VIDEO = "output_with_plates.mp4"


In [10]:
# Colab cell 4 — Inspect dataset layout and create data.yaml for ultralytics
# Expected dataset structure from Roboflow YOLO export: DATA_DIR/train, DATA_DIR/valid, DATA_DIR/test with images + labels folder
from pathlib import Path
import yaml
import os

DATA_DIR = Path(DATA_DIR)
assert DATA_DIR.exists(), f"{DATA_DIR} not found. Make sure dataset extracted."

# Roboflow YOLO exports typically have a 'data.yaml' already. If present, use it; else create one.
existing_yaml = list(DATA_DIR.glob("*.yaml")) + list(DATA_DIR.glob("data.yaml"))
if existing_yaml:
    print("Found dataset yaml:", existing_yaml[0])
    # copy it to root as data.yaml
    import shutil
    shutil.copy(existing_yaml[0], YOLO_DATA_YAML)
    print("Copied to", YOLO_DATA_YAML)
else:
    # create a simple YAML using the train/val/test splits
    train_dir = str(DATA_DIR / "train" / "images")
    val_dir = str(DATA_DIR / "valid" / "images") if (DATA_DIR / "valid").exists() else str(DATA_DIR / "val" / "images")
    test_dir = str(DATA_DIR / "test" / "images") if (DATA_DIR / "test").exists() else None

    # We'll try to infer class names from labels folder
    # Roboflow usually writes a 'labels' folder with .txt files containing class indices; we assume single-class (license plate)
    # We'll default to one class 'license_plate'
    names = ["license_plate"]

    yaml_content = {
        "train": train_dir,
        "val": val_dir,
    }
    if test_dir:
        yaml_content["test"] = test_dir
    yaml_content["nc"] = len(names)
    yaml_content["names"] = names

    with open(YOLO_DATA_YAML, "w") as f:
        yaml.dump(yaml_content, f)

    print("Wrote YOLO data yaml to", YOLO_DATA_YAML)
    print(open(YOLO_DATA_YAML).read())


Found dataset yaml: /content/License-Plate-Recognition-11/data.yaml
Copied to data.yaml


In [12]:
# Colab cell 5 — train YOLO model with ultralytics
# This uses the ultralytics package (YOLOv8/YOLOv9 API compatible)
from ultralytics import YOLO

# Create a YOLO model object from pretrained weights
model = YOLO(MODEL_NAME)  # e.g., yolov8n.pt (downloaded automatically on first use)

# Train
# Save results to a named experiment folder so we can find best.pt
exp_name = "license_plate_yolov8n"
print("Starting training... this may take a while depending on GPU.")
model.train(data=YOLO_DATA_YAML, epochs=EPOCHS, batch=BATCH, imgsz=640, name=exp_name)
print("Training finished. Check the runs directory for results.")


Starting training... this may take a while depending on GPU.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license_plate_yolov8n2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

In [13]:
# Colab cell 6 — pick the best model weights (usually runs/exp_name/weights/best.pt)
import glob
import os
from pathlib import Path

# ultralytics will create something like runs/detect/exp or runs/detect/exp2 etc under project root
weights = list(Path("runs").rglob("best*.pt"))
if not weights:
    # fallback to last.pt
    weights = list(Path("runs").rglob("last*.pt"))
if not weights:
    raise FileNotFoundError("No trained weights found in runs/ — check training output.")
weights = sorted(weights, key=lambda p: p.stat().st_mtime)
model_path = str(weights[-1])
print("Using weights:", model_path)

# load trained model
model = YOLO(model_path)


Using weights: runs/detect/license_plate_yolov8n2/weights/best.pt


In [14]:
# Colab cell 7 — inference on video and OCR per detected plate
# Make sure you have uploaded input.mp4 to Colab (or change INPUT_VIDEO to correct path).
import cv2
import pytesseract
import numpy as np
from pathlib import Path
import math

# pytesseract config — tweak whitelist for your plate format
tess_config = r"--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789- "

INPUT_VIDEO = "/content/demo.mp4"  # replace if needed
OUTPUT_VIDEO = "output_with_plates.mp4"

if not Path(INPUT_VIDEO).exists():
    raise FileNotFoundError(f"{INPUT_VIDEO} not found. Upload your test video to Colab and set INPUT_VIDEO correctly.")

cap = cv2.VideoCapture(INPUT_VIDEO)
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

frame_idx = 0
print("Processing video frames...")

def ocr_plate(img_crop):
    # Preprocess for Tesseract: grayscale, resize, bilateral filter, adaptive threshold
    gray = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
    # scale up small plates
    h_c, w_c = gray.shape[:2]
    scale = max(1, int(200 / max(w_c, h_c)))
    if scale > 1:
        gray = cv2.resize(gray, (w_c*scale, h_c*scale), interpolation=cv2.INTER_CUBIC)
    gray = cv2.bilateralFilter(gray, 9, 75, 75)
    # adaptive threshold
    th = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    # Use pytesseract
    text = pytesseract.image_to_string(th, config=tess_config)
    text = text.strip()
    # basic cleanup
    text = "".join([c for c in text if c.isalnum() or c in "- "])
    return text

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # run model (ultralytics returns results objects)
    results = model.predict(source=frame, conf=0.25, iou=0.45, imgsz=640, max_det=10, verbose=False)  # returns list
    # results is list of Results for each image (we supplied a single frame)
    res = results[0]
    boxes = res.boxes  # Boxes object (xyxy, confidence, cls)
    # convert to numpy arrays
    for i, box in enumerate(boxes):
        xyxy = box.xyxy[0].cpu().numpy()  # [x1,y1,x2,y2]
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        x1, y1, x2, y2 = map(int, xyxy)
        # clamp
        x1, y1 = max(0,x1), max(0,y1)
        x2, y2 = min(w-1,x2), min(h-1,y2)

        # draw bbox
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        label = f"plate {conf:.2f}"
        cv2.putText(frame, label, (x1, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        # crop and OCR
        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:
            plate_text = ""
        else:
            plate_text = ocr_plate(crop)

        # overlay OCR result below bbox
        txt_pos = (x1, y2 + 20)
        if plate_text:
            cv2.putText(frame, plate_text, txt_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        else:
            cv2.putText(frame, "NO_TEXT", txt_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1)

    out.write(frame)

cap.release()
out.release()
print("Saved output video to", OUTPUT_VIDEO)


Processing video frames...
Saved output video to output_with_plates.mp4


In [15]:
# Colab cell 8 — download/check the output
from google.colab import files
print("Now you can download the result file:")
files.download(OUTPUT_VIDEO)


Now you can download the result file:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>